[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/serdna1/ODIR-5K-multi-label-classification/blob/main/experiments/experiment_6/experiment_6.ipynb)

# Download GitHub repository and data
The repo wont be downloading in my local machine, since i already have it there. It will if someone is running this notebook on their own machine, colab, ...

In [ ]:
import os
import zipfile

images_path = 'ODIR-5K-multi-label-classification/data/images/'

# If the experiments folder doesnt exist im not in my local machine
if not os.path.exists('../../experiments'):
    # Clone project repository
    !git clone https://github.com/serdna1/ODIR-5K-multi-label-classification.git

    # Unzip images
    with zipfile.ZipFile(f'{images_path}/train_fov_cc_fov_224.zip', 'r') as zip_ref:
      print("Unzipping training images...")
      zip_ref.extractall(f'{images_path}/train_fov_cc_fov_224')
    !rm ODIR-5K-multi-label-classification/data/images/train_fov_cc_fov_224.zip
    !rm ODIR-5K-multi-label-classification/data/images/train_224.zip

    # Move to the experiment folder
    %cd ODIR-5K-multi-label-classification/experiments/experiment_6/

# Drop some special patients from the dataset
The background of some images is quite different from the rest ones. Therefore they wont be used for training.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

images_path = '../../data/images/train_fov_cc_fov_224'

special_images = ['2174_right.jpg', '2175_left.jpg', '2176_left.jpg', '2177_left.jpg',
                  '2177_right.jpg', '2178_right.jpg', '2179_left.jpg', '2179_right.jpg',
                  '2180_left.jpg', '2180_right.jpg', '2181_left.jpg', '2181_right.jpg',
                  '2182_left.jpg', '2182_right.jpg', '2957_left.jpg', '2957_right.jpg']

# Show one of this special images
image_path = f'{images_path}/{special_images[7]}'
img = Image.open(image_path)
plt.imshow(img)
plt.axis(False)
plt.title('Example of special image')

In [ ]:
import pandas as pd

original_annotations_path = '../../data/annotations/annotations.xlsx'
annotations_path = 'data/annotations.xlsx'

odir_df = pd.read_excel(original_annotations_path)

patients_to_drop_df = odir_df[odir_df['Left-Fundus'].isin(special_images) | odir_df['Right-Fundus'].isin(special_images)]
patients_to_drop_df # This are all the patients with one ore more of this spetial images

In [ ]:
from pathlib import Path

experiment_data_path = 'data/'

print(f'Annotations length before droping the rows with the special images: {len(odir_df)}')

odir_df = odir_df.drop(patients_to_drop_df.index) # Drop the special patients
Path(experiment_data_path).mkdir(parents=True, exist_ok=True) # Create the experiment data folder
odir_df.to_excel(annotations_path) # Save annotations without the special patients

print(f'Annotations length after droping the rows with the special images: {len(odir_df)}')

# Check dataset label distribution

In [ ]:
label_names = ['N','D','G','C','A','H','M','O']

total_label_list = []
for label in label_names:
    total_label = int(odir_df.loc[:, [label]].sum())
    total_label_list.append(total_label)

plt.figure(figsize=(16,4))

bar = plt.bar(label_names, total_label_list)

for rect in bar:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}', ha='center', va='bottom')

plt.xlabel(f'Label')
plt.ylabel(f'No. of images')
plt.title(f'Label distribution')

# Split dataset into train and validation

In [ ]:
os.chdir('../../scripts/')
from utils import split_annotations
os.chdir('../experiments/experiment_6/')

In [ ]:
# Split the original annotations file into train and validation annotations files
# val_ratio = 0.2 (value by default)
split_annotations(annotations_path, experiment_data_path, test_split=False)

# Read the previous three created .xlsx files into dataframes
train_df = pd.read_excel(f'{experiment_data_path}/train_annotations.xlsx')
val_df = pd.read_excel(f'{experiment_data_path}/val_annotations.xlsx')

len(train_df), len(val_df)

# Check train and validation datasets label distribution

In [ ]:
for i, label in enumerate(label_names):
    plt.figure(figsize=(4,4))

    total_label_train = int(train_df.loc[:, [label]].sum())
    total_label_val = int(val_df.loc[:, [label]].sum())

    bar = plt.bar(['Train', 'Val'], [total_label_train, total_label_val])

    for rect in bar:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2.0, height, f'{height:.0f}', ha='center', va='bottom')

    plt.xlabel(f'Dataset')
    plt.ylabel(f'No. of images with label {label}')
    plt.title(f'Distribution of label {label} by dataset')

# Train the model

In [ ]:
%run ../../scripts/train.py -h # Run this line to check the train script arguments

In [26]:
# Arguments that will be passed to the train script
model_name = 'resnet50_dual_v2'
images_path = '../../data/images/train_fov_cc_fov_224'
train_annotations_path = 'data/train_annotations.xlsx'
val_annotations_path = 'data/val_annotations.xlsx'
lr = 0.05
lr_scheduler = 'LinearLR'
batch_size = 32
epochs = 10
patience = 20
experiment_name = 'experiment_6'

In [30]:
%run ../../scripts/train.py --model {model_name} --images_path {images_path} --train_annotations_path {train_annotations_path} --val_annotations_path {val_annotations_path}\
                            --batch_size {batch_size} --epochs {epochs} --patience {patience} --experiment_name {experiment_name} --use_normalization --lr {lr}\
                            --lr_scheduler {lr_scheduler} --num_workers 0

[INFO] Created new resnet50_dual_v1 model.
[INFO] Created SummaryWriter, saving to: ..\runs\resnet50_dual_v1\experiment_5...


  0%|          | 0/10 [00:00<?, ?it/s]

Ep: 0 | t_loss: 0.6268 | t_kappa: 0.0024 | t_f1: 0.2445 | t_auc: 0.4950 | t_final: 0.2473 | v_loss: 0.5463 | v_kappa: 0.0000 | v_f1: 0.0000 | v_auc: 0.5368 | v_final: 0.1789
Validation loss decreased (inf --> 0.546344).  Saving model ...
Ep: 1 | t_loss: 0.4204 | t_kappa: 0.0000 | t_f1: 0.0000 | t_auc: 0.5961 | t_final: 0.1987 | v_loss: 0.4323 | v_kappa: 0.0000 | v_f1: 0.0000 | v_auc: 0.5235 | v_final: 0.1745
Validation loss decreased (0.546344 --> 0.432265).  Saving model ...
Ep: 2 | t_loss: 0.3582 | t_kappa: 0.0000 | t_f1: 0.0000 | t_auc: 0.6826 | t_final: 0.2275 | v_loss: 0.3806 | v_kappa: 0.0000 | v_f1: 0.0000 | v_auc: 0.5357 | v_final: 0.1786
Validation loss decreased (0.432265 --> 0.380577).  Saving model ...
Ep: 3 | t_loss: 0.3731 | t_kappa: 0.0000 | t_f1: 0.0000 | t_auc: 0.7861 | t_final: 0.2620 | v_loss: 0.3752 | v_kappa: 0.0000 | v_f1: 0.0000 | v_auc: 0.5720 | v_final: 0.1907
Validation loss decreased (0.380577 --> 0.375194).  Saving model ...
Ep: 4 | t_loss: 0.3236 | t_kappa:

KeyboardInterrupt: 

Run the next cell to see the metrics trough the epochs using the tensorboard interface

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ../runs

# Test the model with the validation dataset
Since there is no access to the test annotatations (only to the images) the test will be performed on the validation dataset, to be able to see some results.

In [ ]:
%run ../../scripts/test.py -h # Run this line to check the test script arguments

In [ ]:
test_annotations_path = 'data/val_annotations.xlsx' # val annotations are used for test
images_path = '../../data/images/train_fov_cc_fov_224'
model_path = 'outputs/resnet50_dual_v2_experiment_6_model.pth'
ground_truth_path = 'outputs/ground_truth.xlsx'
probs_path = 'outputs/probs.xlsx'

In [ ]:
%run ../../scripts/test.py --model_path {model_path} --model_name {model_name} --images_path {images_path} --test_annotations_path {test_annotations_path} --use_normalization\
                           --ground_truth_path {ground_truth_path} --probs_path {probs_path}

# Run the ODIR_evaluation.py script on test (validation) dataset
This is a file provided along with the data by the challenge. And its supposed to have an example on how to compute the kappa, f1, auc, and final score metrics but i dont think they are well computed here as they are always too good (as shown later, the cualitative results for test are much worst). As a result of this, i use my own version of this metrics (see the function compute_challenge_metrics on the metrics script). Either way i'll run it.

In [ ]:
ground_truth_path = 'outputs/ground_truth.xlsx'
probs_path = 'outputs/probs.xlsx'
probs_csv_path = 'outputs/probs.csv'

probs_df = pd.read_excel(probs_path)
probs_df.to_csv(probs_csv_path, index=False) # The ODIR_evaluation script asks the probs file to be in .csv format

%run ../../scripts/ODIR_evaluation.py {ground_truth_path} {probs_csv_path}

# Cuantitative results
The test script prints some results as can be seen in the output of the previous cell. Now, lets see some more.

In [ ]:
label_names = ['N','D','G','C','A','H','M','O']

ground_truth = pd.read_excel(ground_truth_path).loc[:, label_names].to_numpy()
probs = pd.read_excel(probs_path).loc[:, label_names].to_numpy()

In [ ]:
from sklearn import metrics
os.chdir('../../scripts')
from metrics import plot_confusion_matrices
os.chdir('../experiments/experiment_6/')

# Print test classification report
print('Clasification report:')
print(
    metrics.classification_report(
        ground_truth,
        probs>0.5,
        output_dict=False,
        target_names=label_names,
        zero_division=0
    )
)

# Plot one confusion matrix for each label
plot_confusion_matrices(ground_truth, probs, label_names)

# Cualitative results
One patient with each disease (at least) is predicted. The original images are ploted along with their ground truth and predicted labels.

In [ ]:
import torch
from torchvision import transforms
os.chdir('../../scripts')
from utils import load_model
from predict import pred_and_plot_image
os.chdir('../experiments/experiment_6/')

In [ ]:
model = load_model(model_path, model_name)

test_df = pd.read_excel(test_annotations_path)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

You can run the next cell multiple times to see different results

In [ ]:
for label in label_names:
    patient = test_df[test_df[label] == 1].sample(replace=True).reset_index(drop=True)
    pred_and_plot_image(model=model,
                        left_image_path=f'{images_path}/{patient.at[0, "Left-Fundus"]}',
                        right_image_path=f'{images_path}/{patient.at[0, "Right-Fundus"]}',
                        transform=transform,
                        device=device,
                        label_names=label_names,
                        ground_truth=patient.loc[0, label_names].to_numpy())

# Zip the experiment data folder, the SummaryWriter folder and the experiment outputs
This is necessary in order to be able to download this folders

In [ ]:
# import shutil

# shutil.make_archive('experiment_data_compressed',
#                     format="zip",
#                     root_dir='data/')

# shutil.make_archive('runs_compressed',
#                     format="zip",
#                     root_dir='../runs/')

# shutil.make_archive('outputs_compressed',
#                     format="zip",
#                     root_dir='outputs/')